In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_csv("heart_rates_preprocessing.csv") 
del df['id']

In [ ]:
df_enddate = pd.read_csv("patient_end_date.csv")

In [ ]:
import datetime

def fill_enddate(value):
    if pd.isna(value):
        return datetime.datetime(9999,12,31)
    else:
        return value

df = pd.merge(df, df_enddate, left_on='fitbit_id', right_on='patient_id', how='left')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['end_date'] = pd.to_datetime(df['end_date'])

df['end_date'] = df['end_date'].apply(fill_enddate)
del df['patient_id']
df.rename(columns={'fitbit_id':'patient_id'}, inplace=True)
df = df[df['timestamp'] <= df['end_date']]

In [ ]:
df_patient = pd.read_csv("all_patient.csv")
df = df[df['patient_id'].isin(df_patient['patient_id'])]

In [ ]:
df.sort_values(['patient_id', 'timestamp'], inplace=True)

In [ ]:
import datetime

arr = df.values
my_dict = {}

for i in range(len(arr)):
    if arr[i][0] not in my_dict:
        my_dict[arr[i][0]] = {}
        my_dict[arr[i][0]]['start_date'] = datetime.datetime(year = arr[i][1].year, month = arr[i][1].month ,day = arr[i][1].day)
    my_dict[arr[i][0]]['end_date'] = datetime.datetime(year = arr[i][1].year, month = arr[i][1].month ,day = arr[i][1].day, hour=23, minute=59, second=59)

In [ ]:
my_list = []
for p in tqdm(my_dict, miniters=1, mininterval=1):
    now = my_dict[p]['start_date']
    end_date = my_dict[p]['end_date']
    while now <= end_date:
        my_list.append([p, now])
        now = now + datetime.timedelta(hours = 1)

df_all_date = pd.DataFrame(my_list, columns = ['patient_id', 'timestamp'])
df_all_date['timestamp'] = pd.to_datetime(df_all_date['timestamp'])

df = pd.merge(df_all_date, df, on = ['patient_id', 'timestamp'], how='left')

In [ ]:
from scipy.optimize import curve_fit

def cosine_func(t, A, T, phi, C):
    return A * np.cos(2 * np.pi / T * t + phi) + C

In [ ]:
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


my_list_48_base = []
my_list_168_base = []

for p in my_dict:
    arr = df[df['patient_id']==p].values

    
    for i in range((24 * 2), len(arr), 24):
        hr = arr[i-48:i, 2].reshape(-1)

        time_value = 0
        value_data = []
        time_data = []
        for v in hr:
            if not pd.isna(v):
                value_data.append(v)
                time_data.append(time_value)
            time_value = time_value + 3600

        if len(value_data) <= 3:
            continue

        time_data = np.array(time_data, dtype=float)
        value_data = np.array(value_data, dtype=float)
        
        initial_guess = [50, 86400, 0, np.mean(value_data)] 
        params, _ = curve_fit(cosine_func, time_data, value_data, p0=initial_guess, maxfev = 100000)
        A, T, phi, C = params
        fitted_values = cosine_func(np.array(time_data, dtype=float), A, T, phi, C)
        acrophase = np.max(fitted_values)
        acrophase_time = None
        for t, cos_hr in zip(time_data,fitted_values):
            if cos_hr == acrophase:
                acrophase_time = t
                break
        else:
            acrophase_time = time_data

        mesor = np.mean(fitted_values)
        min_value = np.min(fitted_values)
        amplitude = acrophase - min_value

        X = np.array(time_data).reshape(len(time_data), -1)
        y =fitted_values.reshape(-1, 1)

        model = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
        model.fit(X, y)
        y_pred = model.predict(X)
        acrophase_hour = (acrophase_time/3600)%24
        acrophase_is_abnormal = 0
        if acrophase_hour < 15:
            acrophase_is_abnormal = -1
        elif acrophase_hour > 18:
            acrophase_is_abnormal = 1

        my_list_48_base.append( [p, datetime.datetime(year = arr[i-1][1].year, month = arr[i-1][1].month, \
                                              day = arr[i-1][1].day), \
                        acrophase_hour, acrophase_is_abnormal, acrophase, mesor, min_value, amplitude, T, r2_score(fitted_values, y_pred)] )
                                     
    for i in range((24 * 7) , len(arr), 24):
        hr = arr[i-24 * 7:i, 2].reshape(-1)

        time_value = 0
        value_data = []
        time_data = []
        for v in hr:
            if not pd.isna(v):
                value_data.append(v)
                time_data.append(time_value)
            time_value = time_value + 3600

        if len(value_data) <= 3:
            continue

        time_data = np.array(time_data, dtype=float)
        value_data = np.array(value_data, dtype=float)


        initial_guess = [50, 86400, 0, np.mean(value_data)] 
        params, _ = curve_fit(cosine_func, time_data, value_data, p0=initial_guess, maxfev = 100000)
        A, T, phi, C = params
        fitted_values = cosine_func(np.array(time_data, dtype=float), A, T, phi, C)
        acrophase = np.max(fitted_values)
        acrophase_time = None
        for t, cos_hr in zip(time_data,fitted_values):
            if cos_hr == acrophase:
                acrophase_time = t
                break
        else:
            acrophase_time = time_data

        mesor = np.mean(fitted_values)
        min_value = np.min(fitted_values)
        amplitude = acrophase - min_value

        X = np.array(time_data).reshape(len(time_data), -1)
        y =fitted_values.reshape(-1, 1)

        model = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
        model.fit(X, y)
        y_pred = model.predict(X)
        acrophase_hour = (acrophase_time/3600)%24
        acrophase_is_abnormal = 0
        if acrophase_hour < 15:
            acrophase_is_abnormal = -1
        elif acrophase_hour > 18:
            acrophase_is_abnormal = 1
            

        my_list_168_base.append( [p, datetime.datetime(year = arr[i-1][1].year, month = arr[i-1][1].month, \
                                              day = arr[i-1][1].day), \
                        acrophase_hour, acrophase_is_abnormal, acrophase, mesor, min_value, amplitude, T, r2_score(fitted_values, y_pred)] )

In [ ]:
df_Cosine_Fitting = pd.DataFrame(my_list_48_base, columns=['patient_id', 'timestamp', 'Acrophase_time_base_48_hours', 'Acrophase_is_abnormal_48_hours', 'Acrophase_base_48_hours', 'MESER_base_48_hours', 'Bathyphase_base_48_hours' ,'Amplitude_base_48_hours','Period_base_48_hours', 'goodness_of_fit_base_48_hours'])

In [ ]:
df_Cosine_Fitting2 = pd.DataFrame(my_list_168_base, columns=['patient_id', 'timestamp', 'Acrophase_time_base_168_hours', 'Acrophase_is_abnormal_168_hours', 'Acrophase_base_168_hours', 'MESER_base_168_hours', 'Bathyphase_base_168_hours' ,'Amplitude_base_168_hours','Period_base_168_hours', 'goodness_of_fit_base_168_hours'])

In [ ]:
my_list = []
for p in tqdm(my_dict, miniters=1, mininterval=1):
    now = my_dict[p]['start_date']
    end_date = my_dict[p]['end_date']
    while now <= end_date:
        my_list.append([p, now])
        now = now + datetime.timedelta(days = 1)

df_all_date = pd.DataFrame(my_list, columns = ['patient_id', 'timestamp'])

In [ ]:
df_tmp = pd.merge(df_all_date, df_Cosine_Fitting, on=['patient_id', 'timestamp'], how='left')
df = pd.merge(df_tmp, df_Cosine_Fitting2, on=['patient_id', 'timestamp'], how='left')

In [ ]:
df.to_csv("feature_circadian_rhythm.csv", encoding="utf-8-sig", index=False)